In [162]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to Serial /dev/ttyUSB0 baud=115200 
Ready.


In [3]:
# now try the serial connection version
# ESP32: RX2=Pin17, TX2=Pin16
# BNO055: PS1=high to enable serial mode, SDA=TX, SCL=RX


In [163]:
import machine
u = machine.UART(2, baudrate=115200)  # 8N1


I (44252) uart: ALREADY NULL
I (44252) uart: queue free spaces: 10


In [205]:
import time
def bno055read(reg, n):
    u.write(b"\xAA\x01")
    u.write(chr(reg))
    u.write(chr(n))
    time.sleep_ms(20)
    r = u.read()
    assert r[0] == 0xBB
    assert r[1] == n
    assert len(r) == n + 2
    return r[2:]

def bno055write(reg, r):
    u.write(b"\xAA\x00")
    u.write(chr(reg))
    u.write(chr(len(r)))
    u.write(r)
    time.sleep_ms(20)
    v = u.read()
    assert v == b'\xee\x01', v


In [165]:
u.read() # clear buffer
chipid = bno055read(0x00, 6)
print("CHIP_ID:", hex(chipid[0]), "ACC_ID:", hex(chipid[1]), 
      "MAG_ID:", hex(chipid[2]), "GYR_ID:", hex(chipid[3]), 
      "SW_REV_ID:", hex(chipid[4]), ".", hex(chipid[5])) 
# CHIP_ID:A0 ACC_ID:FB MAG_ID:32 GYR_ID:F SW_REV_ID: 8.3

CHIP_ID: 0xa0 ACC_ID: 0xfb MAG_ID: 0x32 GYR_ID: 0xf SW_REV_ID: 0x8 . 0x3


In [166]:
bno055write(0x3D, b"\x00")   # PWR_MODE
bno055write(0x3B, b"\x00")   # UNIT_SEL, celsius, UDegrees and m/s^2
bno055write(0x3D, b"\x0C")   # back to NDOF mode
print("Temperature", bno055read(0x34, 1)[0])


Temperature 23


In [167]:
# read calibration
bno055write(0x3D, b"\x00")   # PWR_MODE
calib = bno055read(0x55, 22)
bno055write(0x3D, b"\x0C")   # back to NDOF mode
print(calib)  # eg 050006000200000000000000FFFFFEFF0100E8030000


b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\xff\x01\x00\xe8\x03\x00\x00'


In [202]:
print(len(calib), hex(22))


22 0x16


In [206]:

# write calibration
u.read()
calib = b"\x01\x00\x01\x00\x01\x00\xDB\x00\xCC\x00\xE2\x00\xFE\xFF\xFF\xFF\x02\x00\xE8\x03\x38\x02"
bno055write(0x3D, b"\x00")   # PWR_MODE
for i in range(10):          # tends to choke a few times before it gets it
    u.write(b"\xAA\x00\x55\x16"+calib)
    time.sleep_ms(50)
    v = u.read()
    print(v)
    if v == b'\xee\x01':
        break
bno055write(0x3D, b"\x0C")   # back to NDOF mode



b'\xee\x07'
b'\xee\x01'


In [211]:
v = bno055read(0x20, 22)
print(v, hex(22))


b'\xe9\x03\xae\x04K\xc0B\x00\xfc\xff\x01\x00\xf8\xffw\x00\x00\x003\xfc\x1a?' 0x16


In [247]:
# set the device to continually ping every 10ms for a reading
import machine
timer0 = machine.Timer(0)
def timer0callback(ltimer):
    u.write(b"\xAA\x01\x20\x16")
timer0.init(period=10, mode=machine.Timer.PERIODIC, callback=timer0callback)


In [378]:
timer0.deinit()
time.sleep(1)
u.read()
timer0.init(period=10, mode=machine.Timer.PERIODIC, callback=timer0callback)


In [379]:
#timer0.deinit()
print(u.read())


b'\xee\x06\xee\x06\xee\x06\xee\x06\xee\x06\xee\x06\xee\x06\xee\x06\xee\x06\xee\x06\xee\x06\xee\x06\xee\x06\xee\x06\xee\xee\n\xee\x07\xbb\x16 \x04x\xfc;\xc0\xbc\xff\x00\x00\x03\x00\x03\x00~\x00\x01\x004\xfc\x1a7\xbb\x16 \x04x\xfc;\xc0\xbc\xff\x01\x00\x05\x00\x07\x00~\x00\x01\x004\xfc\x1a7\xbb\x16 \x04x\xfc;\xc0\xbc\xff\x01\x00\x01\x00\x02\x00~\x00\x01\x004\xfc\x1a7\xbb\x16 \x04x\xfc;\xc0\xbc\xff\x00\x00\x02\x00\xfd\xff~\x00\x01\x004\xfc\x1a7\xbb\x16 \x04x\xfc;\xc0\xbc\xff\x00\x00\x05\x00\x01\x00~\x00\x01\x004\xfc\x1a7\xbb\x16 \x04x\xfc;\xc0\xbc\xff\x00\x00\x03\x00\x06\x00~\x00\x01\x004\xfc\x1a7\xbb\x16 \x04x\xfc;\xc0\xbc\xff\x00\x00\x02\x00\x05\x00~\x00\x01\x004\xfc\x1a7\xbb\x16 \x04x\xfc;\xc0\xbc\xff\x00\x00\x00\x00\x06\x00~\x00\x01\x004\xfc\x1a7\xbb\x16 \x04x\xfc;\xc0\xbc\xff\x00\x00\x01\x00\xfd\xff~\x00\x01\x004\xfc\x1a7\xbb\x16 \x04x\xfc;\xc0\xbc\xff\x00\x00\x02\x00\x00\x00~4\xfc\x1b7\xbb\x16 \x04'


In [350]:
# ongoing function agregating bytes by groups of 24
b055buff = bytearray(24)
mb055buff = memoryview(b055buff)
b055buffN = 0
def bno055getorientbuffer():
    global b055buffN
    while True:
        if b055buffN == 24:
            b055buffN = 0
        n = u.readinto(mb055buff[b055buffN:])
        if n == 0:
            return False
        b055buffN += n
        if b055buffN != 24:
            return False
        if b055buff[0] == 0xBB:
            return True
        for i in range(1,24):
            if b055buff[i] == 0xBB:
                b055buffN = 24-i
                mb055buff[:b055buffN] = mb055buff[i:]
        else:
            mb055buffN = 0
            


True
True
True
True
True
True
True
True
True
True
False
False
False
True
False
False
False
False
False
True


In [408]:
print(b055buff)
print(u.read())

bytearray(b'\x06\xee\x06\xee\x06\xee\x06\xee\x06\xee\x06\xee\x06\xee\x06\xee\x06\xee\x06\xee\x06\xee\x06\xee')
b's\x00\xf5\xff3\xfc\x1b\xf3\xbb\x16\xa9\x03\xbf\xe8\x7f\xc4\xfb\xfe\xfa\xff\x06\x00\x02\x00s\xbb\x16\xa9\x03\xbf\xe8\x7f\xc4\xfb\xfe\xfa\xff\x06\x00\x02\x00s\x00\xf5\xff3\xfc\x1b\xf3\xbb\x16\xa9\x03\xbf\xe8\x7f\xc4\xfb\xfe\xfa\xff\x05\x00\xf8\xffs\x00\xf5\xff3\xfc\x1b\xf3\xbb\x16\xa9\x03\xbf\xe8\x7f\xc4\xfb\xfe\xf6\xff\x06\x00\x08\x00s\x00\xf5\xff3\xfc\x1b\xf3\xbb\x16\xa9\x03\xbf\xe8\x7f\xc4\xfb\xfe\xf6\xff\x04\x00\x0c\x00s\x00\xf5\xff3\xfc\x1b\xf3\xbb\x16\xa9\x03\xbf\xe8\x7f\xc4\xfb\xfe\xfb\xff\x06\x00\x00\x00s\x00\xf5\xff3\xfc\x1b\xf3\xbb\x16\xa9\x03\xbf\xe8\x7f\xc4\xfb\xfe\xfa\xff\x07\x00\x00\x00s\x00\xf5\xff3\xfc\x1b\xf3\xbb\x16\xa9\x03\xbf\xe8\x7f\xc4\xfb\xfe\xfa\xff\x06\x00\xfd\xffs\x00\xf5\xff3\xfc\x1b\xf3\xbb\x16\xa9\x03\xbf\xe8\x7f\xc4\xfb\xfe\xf9\xff\x06\x00\x01\x00s\x00\xf5\xff3\xfc\x1b\xf3\xbb\x16\xa9\x03\xbf\xe8\x7f\xc4\xfb\xfe\xfa\xff\x06\x00\x02\x00s\x00\xf5\xf

In [411]:
%capture --quiet bnodata.txt
import ustruct
for i in range(2000):
    if bno055getorientbuffer():
        qw, qx, qy, qz = ustruct.unpack("<hhhh", mb055buff[2:10])
        accx, accy, accz = ustruct.unpack("<hhh", mb055buff[10:16])
        gx, gy, gz = ustruct.unpack("<hhh", mb055buff[16:24])
        print(gx, gy, gz)
    time.sleep_ms(2)


496 lines captured

In [25]:
import machine
i2c = machine.I2C(scl=machine.Pin(4), sda=machine.Pin(5), freq=100000)  #D2,D1
# NodeMCU D4 is Pin(2) the light
# D7 is Pin(13)
# D1 is Pin(5), D2 is Pin(4)

In [38]:
print(i2c.scan())

[40]


In [39]:
help(i2c)

object <I2C> is of type I2C
  init -- <function>
  scan -- <function>
  start -- <function>
  stop -- <function>
  readinto -- <function>
  write -- <function>
  readfrom -- <function>
  readfrom_into -- <function>
  writeto -- <function>
  readfrom_mem -- <function>
  readfrom_mem_into -- <function>
  writeto_mem -- <function>


In [43]:
k = i2c.readfrom_mem(0x28, 0x00, 6)
print("BNO055 sensor SW_REV_ID: %s.%s" %(hex(k[4]), hex(k[5])))

i2c.writeto_mem(0x28, 0x3D, b'\x00')     # config mode
i2c.writeto_mem(0x28, 0x3E, b'\x00')     # PWR_MODE, normal
i2c.writeto_mem(0x28, 0x3B, b'\x00')     # UNIT_SEL, celsius, UDegrees and m/s^2
i2c.writeto_mem(0x28, 0x3D, b'\x0c')     # back to NDOF mode


BNO055 sensor SW_REV_ID: 0x8.0x3


In [48]:
import ustruct, math
def BNO055calibstat():
    calibstat = i2c.readfrom_mem(0x28, 0x35, 1)[0]
    print("sys:", (calibstat>>6)&0x03, "gyr:", (calibstat>>4)&0x03, "acc:", (calibstat>>2)&0x03, "mag:", calibstat&0x03)
    return calibstat

def BNO055quat():  # returns qw, qx, qy, qz
    return ustruct.unpack("<hhhh", i2c.readfrom_mem(0x28, 0x20, 8))
    #accx, accy, accz = ustruct.unpack("<hhh", i2c.readfrom_mem(0x28, 0x28, 6))
    #gravx, gravy, gravz = ustruct.unpack("<hhh", i2c.readfrom_mem(0x28, 0x2E, 6))
    
def BNO055pitchrollorient():  
    q0, q1, q2, q3 = ustruct.unpack("<hhhh", i2c.readfrom_mem(0x28, 0x20, 8))
    riqsq = q0*q0 + q1*q1 + q2*q2 + q3*q3 
    iqsq = 1/riqsq 
    
    r02 = q0*q2*2 * iqsq
    r13 = q1*q3*2 * iqsq
    sinpitch = r13 - r02

    r01 = q0*q1*2 * iqsq
    r23 = q2*q3*2 * iqsq 
    sinroll = r23 + r01 
     
    r00 = q0*q0*2 * iqsq
    r11 = q1*q1*2 * iqsq
    r03 = q0*q3*2 * iqsq
    r12 = q1*q2*2 * iqsq
    a00=r00 - 1 + r11   
    a01=r12 + r03  
    rads = math.atan2(a00, -a01) 
    northorient = 180 - math.degrees(rads) 
    return math.degrees(math.asin(sinpitch)), math.degrees(math.asin(sinroll)), northorient


In [50]:
import time
for i in range(50):
    print(BNO055pitchrollorient())
    time.sleep(1.0)

(10.8158, -44.0199, 61.588)
(10.8238, -47.5484, 62.3141)
(20.7014, -60.9549, 54.0808)
(56.971, -33.0013, 43.9023)
(52.1493, -18.3065, 342.545)
(46.3006, -35.2649, 17.5424)
(42.2377, -27.7535, 16.5857)
.(8.33827, -48.6177, 7.79858)
(3.4224, -35.0811, 17.7718)
(2.68431, -75.0881, 29.6096)
(2.22569, -85.1109, 33.5448)
(4.20045, -85.6618, 33.629)
(2.58186, -66.25, 28.2278)
.(-28.3882, -59.442, 23.9154)
(-35.7429, -51.7838, 25.7964)
(28.4852, -51.7594, 30.1144)
(30.6728, -49.8863, 29.4197)
(10.4922, -70.1416, 25.254)
(-16.8348, -58.7813, 344.523)
(12.2092, -69.8632, 354.49)
.(17.637, -47.6771, 349.497)
(13.2955, -57.9992, 359.52)
(14.5041, -55.6848, 1.68188)
(15.4812, -53.5235, 1.039)
(15.467, -52.7755, 359.598)
(15.0609, -52.7346, 358.906)
.(15.1284, -52.8254, 359.245)
(15.2328, -55.3978, 358.871)
(14.1985, -54.5009, 357.564)
(13.7786, -57.1812, 357.214)
(14.3266, -59.1064, 357.278)
(18.8258, -71.0725, 2.16046)
(12.4669, -44.7047, 4.23883)
.(9.14068, -48.523, 2.27264)
(7.94079, -50.0397, 0

In [13]:
help(i2c)

object <I2C> is of type I2C
  init -- <function>
  scan -- <function>
  start -- <function>
  stop -- <function>
  readinto -- <function>
  write -- <function>
  readfrom -- <function>
  readfrom_into -- <function>
  writeto -- <function>
  readfrom_mem -- <function>
  readfrom_mem_into -- <function>
  writeto_mem -- <function>


In [14]:
i2c.start()